In [ ]:
# 인물별 대사 분리 후 파일 출력
import os, re
os.chdir('/content/drive/MyDrive/Data/')
a = open('Day-5-data01.txt').read()
b = re.findall('^.\s+(.+)/.+:\s+(.+)', a, flags=re.MULTILINE)
os.makedirs('./debates', exist_ok=True)
for i, j in b:
  open('./debates/' + i + '.txt', 'a').write(j + '\n')

In [ ]:
# 파일별 형태소 분석 후 파일 출력
# ! pip install konlpy
from konlpy.tag import Kkma
import os
os.makedirs('./morphs', exist_ok=True)
for fileName in os.listdir('./debates'):
  if fileName.endswith('.txt'):
    text = open('./debates/' + fileName).read()
    morphs = '\n'.join('/'.join(w) for w in Kkma().pos(text))
    open('./morphs/' + fileName.replace('.txt', '.mor'), 'w').write(morphs)

In [ ]:
# 한글 폰트 Google Colab 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 여러 파일 하나의 리스트로 만들기
import os
os.chdir('/content/drive/MyDrive/Data/')
texts = []
for fileName in os.listdir('./morphs'):
  if fileName.endswith('.mor'):
    texts.extend(open('./morphs/' + fileName).read().split())

In [ ]:
# 한글 폰트 지정 및 고빈도 라인차트 생성
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import nltk
nltk.FreqDist(texts).plot(30)

In [ ]:
# 체언 및 용언 고빈도 라인차트 생성
import re
nltk.FreqDist(w for w in texts if re.search('/V|/N', w)).plot(30)

In [ ]:
# 일반명사 및 고유명사 고빈도 워드클라우드
import wordcloud
from collections import Counter
Freq = Counter(w for w in texts if '/NNG' in w or '/NNP' in w)
wc = wordcloud.WordCloud(background_color='white', max_words=50, 
                         font_path='NanumBarunGothic').generate_from_frequencies(Freq)
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.imshow(wc)
plt.axis('off')

In [ ]:
# 파일별 빈도 추출 TDM 만들기
import os, unicodedata
from collections import Counter
os.chdir('/content/drive/MyDrive/Data/')
Freq = {}
term = unicodedata.normalize('NFD', '편상욱')
for fileName in os.listdir('./morphs'): 
  if fileName.endswith('.mor') and not fileName.startswith(term):
    Freq[fileName.replace('.mor', '')] =  Counter(open('./morphs/' + 
                                                       fileName).read().split())
import pandas as pd
df = pd.DataFrame(Freq)

In [ ]:
# Tree Map 만들기: 인물별 비교
import plotly.express as px
fig = px.treemap(df.stack().reset_index(),
                 path=['level_1', 'level_0'], values=0)
fig.show()

In [ ]:
# Tree Map 만들기: 형태소별 비교
import plotly.express as px
df_new = df.stack().reset_index()
fig = px.treemap(df_new[df_new['level_0'].str.contains("/NNG|/NNP")],
                 path=['level_0', 'level_1'], values=0)
fig.show()